## CS 533 Group Project
Authors: Malia Barker, Atharva Pargaonkar, Jack Cunningham, Rubal Goyal, Shrutee Dwa

### Importing python tools and the data

In [101]:
# Import the required tools and libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [102]:
# Import the movies dataframe
movies_path = './ml-25m/movies.csv'
movies_df = pd.read_csv(movies_path)
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [103]:
# Import the ratings dataframe
ratings_path = './ml-25m/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


In [104]:
# Import the tags dataframe
tags_path = './ml-25m/tags.csv'
tags_df = pd.read_csv(tags_path)
tags_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093360 entries, 0 to 1093359
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   userId     1093360 non-null  int64 
 1   movieId    1093360 non-null  int64 
 2   tag        1093344 non-null  object
 3   timestamp  1093360 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 33.4+ MB


<hr>

### Merging datasets

First, let's look at merging the ratings and the tags datasets.

In [105]:
ratings_and_tags = pd.merge(ratings_df, tags_df, on=['userId', 'movieId'], how='left')

In [106]:
print(f'The ratings dataset has {ratings_df.shape[0]} total ratings')
print(f'The tags dataset has {tags_df.shape[0]} total tags')
print(f'In the ratings dataset, there are {len(ratings_df["movieId"].unique())} unique movies')
print(f"In the tags dataset, there are {len(tags_df['movieId'].unique())} unique movies")
print(f"When combining the ratings and tags datasets, there are {ratings_and_tags['tag'].isna().sum()} ratings that do not have any tags (are NaN)")

The ratings dataset has 25000095 total ratings
The tags dataset has 1093360 total tags
In the ratings dataset, there are 59047 unique movies
In the tags dataset, there are 45251 unique movies
When combining the ratings and tags datasets, there are 24789371 ratings that do not have any tags (are NaN)


There are a lot less tags than there are actual ratings, so there is a high number of null values in this dataset. Combining the tags and ratings like this, resulting in such a large number of columns and ultimately making it so the models will run very slow is probably not ideal. Instead, it would be much better to simply group the tags by userId and movieId, then add the tags as a string to the ratings by userId and movieId. This will result in retaining the shape of the combined dataset, and still allowing the tags to be in there. As for ratings that do not have tags, a string of 'No tags' will be put in as a placeholder.

In [107]:
grouped_tags = tags_df.groupby(['userId', 'movieId'])['tag'].agg(list).reset_index()

In [108]:
grouped_tags['tag'] = grouped_tags['tag'].apply(lambda x: ', '.join(map(str, x)))

In [109]:
grouped_tags

,userId,movieId,tag
0,3,260,"classic, sci-fi"
1,4,1732,"dark comedy, great dialogue"
2,4,7569,so bad it's good
3,4,44665,unreliable narrators
4,4,115569,tense
...,...,...,...
305351,162521,8874,cornetto triolgy
305352,162521,51255,cornetto trilogy
305353,162521,66934,"Nathan Fillion, Neil Patrick Harris"
305354,162521,103341,cornetto trilogy


Now that we have the tags grouped by userID and movieID, let's add them to the ratings dataframe and fill any rows without ratings with the placeholder 'No tags'.

In [110]:
ratings_df = pd.merge(ratings_df, grouped_tags, on=['userId', 'movieId'], how='left')

In [113]:
ratings_df['tag'] = ratings_df['tag'].fillna(str('No tags'))

In [114]:
print(ratings_df.head())
print(ratings_df.info())
print(ratings_df.isna().sum())

   userId  movieId  rating   timestamp      tag
0       1      296     5.0  1147880044  No tags
1       1      306     3.5  1147868817  No tags
2       1      307     5.0  1147868828  No tags
3       1      665     5.0  1147878820  No tags
4       1      899     3.5  1147868510  No tags
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
 4   tag        object 
dtypes: float64(1), int64(3), object(1)
memory usage: 953.7+ MB
None
userId       0
movieId      0
rating       0
timestamp    0
tag          0
dtype: int64


Now let's look into merging ratings and movies.

Merging to ratings and movies dataset on each instance would not work because the ratings dataset is too large and there would be way too many unique combinations of ratings and movies for the scope of this project.

To handle merging movies and ratings, we need to get some more concise values of ratings to add to the dataset.

We can do this by grouping the ratings by movie ID, then getting some statistics from the groups and ratings such as mean, median, mode, min, max, the quartiles, and more.


In [21]:
ratings_df.describe()

,userId,movieId,rating,timestamp
count,2.500010e+07,2.500010e+07,2.500010e+07,2.500010e+07
mean,8.118928e+04,2.138798e+04,3.533854e+00,1.215601e+09
std,4.679172e+04,3.919886e+04,1.060744e+00,2.268758e+08
min,1.000000e+00,1.000000e+00,5.000000e-01,7.896520e+08
25%,4.051000e+04,1.196000e+03,3.000000e+00,1.011747e+09
50%,8.091400e+04,2.947000e+03,3.500000e+00,1.198868e+09
75%,1.215570e+05,8.623000e+03,4.000000e+00,1.447205e+09
max,1.625410e+05,2.091710e+05,5.000000e+00,1.574328e+09


In [38]:
ratings_stats = ratings_df.groupby('movieId')['rating'].agg(['mean', 'median', 'min', 'max', 'std', 'var', 'count']).reset_index()
ratings_stats['q25'] = ratings_df.groupby('movieId')['rating'].quantile(0.25).reset_index()['rating']
ratings_stats['q50'] = ratings_df.groupby('movieId')['rating'].quantile(0.50).reset_index()['rating']
ratings_stats['q75'] = ratings_df.groupby('movieId')['rating'].quantile(0.75).reset_index()['rating']
ratings_stats['mode'] = ratings_df.groupby('movieId')['rating'].agg(lambda x: x.mode().iat[0]).reset_index()['rating']
rename_cols_dict = {}
for col in ratings_stats.columns:
    if col != 'movieId':
        new_col_name = 'ratings_' + str(col) 
        rename_cols_dict[col] = new_col_name
ratings_stats = ratings_stats.rename(columns=rename_cols_dict)
ratings_stats

,movieId,ratings_mean,ratings_median,ratings_min,ratings_max,ratings_std,ratings_var,ratings_count,ratings_q25,ratings_q50,ratings_q75,ratings_mode
0,1,3.893708,4.0,0.5,5.0,0.921552,0.849258,57309,3.5,4.0,4.5,4.0
1,2,3.251527,3.0,0.5,5.0,0.959851,0.921315,24228,3.0,3.0,4.0,3.0
2,3,3.142028,3.0,0.5,5.0,1.008443,1.016957,11804,3.0,3.0,4.0,3.0
3,4,2.853547,3.0,0.5,5.0,1.108531,1.228841,2523,2.0,3.0,4.0,3.0
4,5,3.058434,3.0,0.5,5.0,0.996611,0.993234,11714,2.5,3.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
59042,209157,1.500000,1.5,1.5,1.5,NaN,NaN,1,1.5,1.5,1.5,1.5
59043,209159,3.000000,3.0,3.0,3.0,NaN,NaN,1,3.0,3.0,3.0,3.0
59044,209163,4.500000,4.5,4.5,4.5,NaN,NaN,1,4.5,4.5,4.5,4.5
59045,209169,3.000000,3.0,3.0,3.0,NaN,NaN,1,3.0,3.0,3.0,3.0


Now that we have some basic descriptive stats for ratings, we can join these with the movies dataset on movie ID.

In [39]:
movies_df = pd.merge(movies_df, ratings_stats, on="movieId", how="inner")
movies_df.head()

,movieId,title,genres,ratings_mean,ratings_median,ratings_min,ratings_max,ratings_std,ratings_var,ratings_count,ratings_q25,ratings_q50,ratings_q75,ratings_mode
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.893708,4.0,0.5,5.0,0.921552,0.849258,57309,3.5,4.0,4.5,4.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.251527,3.0,0.5,5.0,0.959851,0.921315,24228,3.0,3.0,4.0,3.0
2,3,Grumpier Old Men (1995),Comedy|Romance,3.142028,3.0,0.5,5.0,1.008443,1.016957,11804,3.0,3.0,4.0,3.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.853547,3.0,0.5,5.0,1.108531,1.228841,2523,2.0,3.0,4.0,3.0
4,5,Father of the Bride Part II (1995),Comedy,3.058434,3.0,0.5,5.0,0.996611,0.993234,11714,2.5,3.0,4.0,3.0
